<a href="https://colab.research.google.com/github/prof-nuduls/EE628HW/blob/main/HW9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz

In [3]:
!tar xf spark-3.5.1-bin-hadoop3.tgz

In [4]:
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [5]:
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [6]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

from google.colab import files

In [7]:
"""
HW9.py
Author: Derick Miller
Class: AAI627
Date: April 14, 2024
Description:
"""
from pyspark.sql import SparkSession,Window
from pyspark.sql.functions import concat, col,lit,row_number
from pyspark.ml.feature import VectorAssembler,StringIndexer,VectorIndexer
from pyspark.ml.classification import LogisticRegression, GBTClassifier,RandomForestClassifier
from pyspark.ml.regression import LinearRegression,GBTRegressor,RandomForestRegressor,DecisionTreeRegressor
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
import numpy as np

import statistics
import pandas as pd
from sklearn.preprocessing import MinMaxScaler,StandardScaler


In [8]:
data = pd.read_csv("output1.txt", sep="|", header=None)
# Add column names based on your data description
column_names = ["UsrID","TrackID", "AlbumRating", "ArtistRating", "NumGenres","MaxGenre", "MinGenre", "SumGenre", "VarGenre","MeanGenre" ]
data.columns = column_names

ground_truth = pd.read_csv("test2_new.txt",sep='|',header=None)
column_names_2 = ['UsrID','TrackID','G_truth']
ground_truth.columns=column_names_2
print(ground_truth)

       UsrID  TrackID  G_truth
0     200031    30877        1
1     200031     8244        1
2     200031   130183        0
3     200031   198762        0
4     200031    34503        1
...      ...      ...      ...
5995  212234   137371        0
5996  212234    42375        0
5997  212234   277867        1
5998  212234    83093        1
5999  212234   239143        1

[6000 rows x 3 columns]


In [9]:
# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Normalize each column separately
normalized_data1 = pd.DataFrame(scaler.fit_transform(data.iloc[:, 2:]), columns=column_names[2:])
normalized_data = pd.concat([data[['UsrID', 'TrackID']], normalized_data1], axis=1)

print(normalized_data)

         UsrID  TrackID  AlbumRating  ArtistRating  NumGenres  MaxGenre  \
0       199810   208019          0.0           0.0   0.000000       0.0   
1       199810    74139          0.0           0.0   0.066667       0.8   
2       199810     9903          0.0           0.0   0.000000       0.0   
3       199810   242681          0.0           0.0   0.000000       0.0   
4       199810    18515          0.0           0.7   0.000000       0.0   
...        ...      ...          ...           ...        ...       ...   
119995  249010    72192          0.0           0.0   0.000000       0.0   
119996  249010    86104          0.0           0.0   0.000000       0.0   
119997  249010   186634          0.9           0.9   0.000000       0.0   
119998  249010   293818          0.0           0.0   0.000000       0.0   
119999  249010   262811          0.9           0.9   0.000000       0.0   

        MinGenre  SumGenre  VarGenre  MeanGenre  
0            0.0  0.000000       0.0        0.0  

In [10]:
train_data = pd.merge(ground_truth, normalized_data, on=['UsrID', 'TrackID'], how='inner')
feature_columns = train_data.columns[3:]
feature_columns_list = feature_columns.tolist()
#print(train_data)
train_data = spark.createDataFrame(train_data)
# Assemble features into a vector column
assembler = VectorAssembler(inputCols=feature_columns_list, outputCol="features")
transformed_data = assembler.transform(train_data)

# Specify the label column
train_data = transformed_data.withColumnRenamed("G_truth", "label")
train_data.show()



+------+-------+-----+-----------+------------+-------------------+--------+--------+-------------------+--------+------------------+--------------------+
| UsrID|TrackID|label|AlbumRating|ArtistRating|          NumGenres|MaxGenre|MinGenre|           SumGenre|VarGenre|         MeanGenre|            features|
+------+-------+-----+-----------+------------+-------------------+--------+--------+-------------------+--------+------------------+--------------------+
|200031|  30877|    1|        0.9|         0.5|0.13333333333333333|     0.9|     0.8| 0.1954022988505747|    0.01|              0.85|[0.9,0.5,0.133333...|
|200031|   8244|    1|        0.9|         0.0|0.13333333333333333|     0.9|     0.8| 0.1954022988505747|    0.01|              0.85|[0.9,0.0,0.133333...|
|200031| 130183|    0|        0.0|         0.0|                0.0|     0.0|     0.0|                0.0|     0.0|               0.0|           (8,[],[])|
|200031| 198762|    0|        0.0|         0.0|0.06666666666666667|   

In [11]:
# LOGISTIC REGRESSION
lr = LinearRegression(maxIter=25, regParam=0.3, elasticNetParam=0.8)
model = lr.fit(train_data)


# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(model.coefficients))
print("Intercept: " + str(model.intercept))

Coefficients: [0.0,0.21493366635291672,0.0,0.0,0.0,0.0,0.0,0.0]
Intercept: 0.4247753661131433


In [12]:
test_data = spark.createDataFrame(normalized_data)
feature_columns = normalized_data.columns[2:]
feature_columns_list = feature_columns.tolist()
# Assemble features into a vector column
assembler = VectorAssembler(inputCols=feature_columns_list, outputCol="features")
test_data = assembler.transform(test_data)
test_data.show()
print(feature_columns)

+------+-------+-----------+------------------+-------------------+--------+--------+-------------------+--------+---------+--------------------+
| UsrID|TrackID|AlbumRating|      ArtistRating|          NumGenres|MaxGenre|MinGenre|           SumGenre|VarGenre|MeanGenre|            features|
+------+-------+-----------+------------------+-------------------+--------+--------+-------------------+--------+---------+--------------------+
|199810| 208019|        0.0|               0.0|                0.0|     0.0|     0.0|                0.0|     0.0|      0.0|           (8,[],[])|
|199810|  74139|        0.0|               0.0|0.06666666666666667|     0.8|     0.8|0.09195402298850575|     0.0|      0.8|[0.0,0.0,0.066666...|
|199810|   9903|        0.0|               0.0|                0.0|     0.0|     0.0|                0.0|     0.0|      0.0|           (8,[],[])|
|199810| 242681|        0.0|               0.0|                0.0|     0.0|     0.0|                0.0|     0.0|      0.0|

In [13]:
predictions = model.transform(test_data)
# Select the columns of interest
# Select the columns of interest and concatenate UsrID and TrackID into a new column named USRID_TRACKID
predictions = predictions.withColumn("TrackID_conc", concat(col("UsrID"), lit('_'), col("TrackID")))
predictions1 = predictions.toPandas()

grouped = predictions1.groupby('UsrID')['prediction'].apply(np.array)

# Iterate over each group and select the top 3 indices with the largest values
suggestions = []
for group in grouped:
    indices = np.argsort(group)
    largest_indices = indices[-3:]
    user_suggestions = np.zeros_like(group)
    user_suggestions[largest_indices] = 1
    suggestions.append(user_suggestions)

  # Concatenate the suggestions for all users into a single array
suggestions_array = np.concatenate(suggestions)

# Create a new DataFrame with TrackID, UsrID, and suggestions
suggestions_df = pd.DataFrame({'UsrID': predictions1['UsrID'],'TrackID': predictions1['TrackID'], 'Suggestion': suggestions_array})

# Display the resulting DataFrame
print(suggestions_df)


suggestions_df['UserID_TrackID'] = suggestions_df['UsrID'].astype(str) + '_' + suggestions_df['TrackID'].astype(str)

# Create DataFrame with 'UserID_TrackID' and 'Suggestion' columns
df = pd.DataFrame({'TrackID': suggestions_df['UserID_TrackID'], 'Predictor': suggestions_df['Suggestion']})

# Convert 'Suggestion' column to integer type
df['Predictor'] = df['Predictor'].astype(int)
df.head(12)

         UsrID  TrackID  Suggestion
0       199810   208019         0.0
1       199810    74139         0.0
2       199810     9903         0.0
3       199810   242681         1.0
4       199810    18515         1.0
...        ...      ...         ...
119995  249010    72192         0.0
119996  249010    86104         0.0
119997  249010   186634         1.0
119998  249010   293818         0.0
119999  249010   262811         1.0

[120000 rows x 3 columns]


,TrackID,Predictor
0,199810_208019,0
1,199810_74139,0
2,199810_9903,0
3,199810_242681,1
4,199810_18515,1
5,199810_105760,1
6,199812_276940,0
7,199812_142408,1
8,199812_130023,1
9,199812_29189,0


In [14]:
df.to_csv('predictions_LR.csv', index=False)

In [15]:
## RANDOM FOREST
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(train_data)
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=5).fit(train_data)

# Train a RandomForest model.
rf = RandomForestRegressor(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

# Train model.  This also runs the indexers.
model = pipeline.fit(train_data)

predictions = model.transform(test_data)
# Select the columns of interest
predictions1 = predictions.toPandas()

grouped = predictions1.groupby('UsrID')['prediction'].apply(np.array)

# Iterate over each group and select the top 3 indices with the largest values
suggestions = []
for group in grouped:
    indices = np.argsort(group)
    largest_indices = indices[-3:]
    user_suggestions = np.zeros_like(group)
    user_suggestions[largest_indices] = 1
    suggestions.append(user_suggestions)

  # Concatenate the suggestions for all users into a single array
suggestions_array = np.concatenate(suggestions)

# Create a new DataFrame with TrackID, UsrID, and suggestions
suggestions_df = pd.DataFrame({'UsrID': predictions1['UsrID'],'TrackID': predictions1['TrackID'], 'Suggestion': suggestions_array})

# Display the resulting DataFrame
print(suggestions_df)


suggestions_df['UserID_TrackID'] = suggestions_df['UsrID'].astype(str) + '_' + suggestions_df['TrackID'].astype(str)

# Create DataFrame with 'UserID_TrackID' and 'Suggestion' columns
df = pd.DataFrame({'TrackID': suggestions_df['UserID_TrackID'], 'Predictor': suggestions_df['Suggestion']})

# Convert 'Suggestion' column to integer type
df['Predictor'] = df['Predictor'].astype(int)
df.head(12)

         UsrID  TrackID  Suggestion
0       199810   208019         0.0
1       199810    74139         1.0
2       199810     9903         0.0
3       199810   242681         0.0
4       199810    18515         1.0
...        ...      ...         ...
119995  249010    72192         0.0
119996  249010    86104         0.0
119997  249010   186634         1.0
119998  249010   293818         0.0
119999  249010   262811         1.0

[120000 rows x 3 columns]


,TrackID,Predictor
0,199810_208019,0
1,199810_74139,1
2,199810_9903,0
3,199810_242681,0
4,199810_18515,1
5,199810_105760,1
6,199812_276940,0
7,199812_142408,1
8,199812_130023,1
9,199812_29189,0


In [16]:
df.to_csv('predictions_RF.csv', index=False)

In [17]:
## GBT
# Train a GBT model.
gbt = GBTRegressor(labelCol="indexedLabel", featuresCol="indexedFeatures", maxIter=100)

# Chain indexers and GBT in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, gbt])

# Train model.  This also runs the indexers.
model = pipeline.fit(train_data)

predictions = model.transform(test_data)
# Select the columns of interest
# Select the columns of interest and concatenate UsrID and TrackID into a new column named USRID_TRACKID
predictions = predictions.withColumn("TrackID_conc", concat(col("UsrID"), lit('_'), col("TrackID")))
predictions1 = predictions.toPandas()

grouped = predictions1.groupby('UsrID')['prediction'].apply(np.array)

# Iterate over each group and select the top 3 indices with the largest values
suggestions = []
for group in grouped:
    indices = np.argsort(group)
    largest_indices = indices[-3:]
    user_suggestions = np.zeros_like(group)
    user_suggestions[largest_indices] = 1
    suggestions.append(user_suggestions)

  # Concatenate the suggestions for all users into a single array
suggestions_array = np.concatenate(suggestions)

# Create a new DataFrame with TrackID, UsrID, and suggestions
suggestions_df = pd.DataFrame({'UsrID': predictions1['UsrID'],'TrackID': predictions1['TrackID'], 'Suggestion': suggestions_array})

# Display the resulting DataFrame
print(suggestions_df)


suggestions_df['UserID_TrackID'] = suggestions_df['UsrID'].astype(str) + '_' + suggestions_df['TrackID'].astype(str)

# Create DataFrame with 'UserID_TrackID' and 'Suggestion' columns
df = pd.DataFrame({'TrackID': suggestions_df['UserID_TrackID'], 'Predictor': suggestions_df['Suggestion']})

# Convert 'Suggestion' column to integer type
df['Predictor'] = df['Predictor'].astype(int)
df.head(12)

         UsrID  TrackID  Suggestion
0       199810   208019         0.0
1       199810    74139         1.0
2       199810     9903         0.0
3       199810   242681         0.0
4       199810    18515         1.0
...        ...      ...         ...
119995  249010    72192         0.0
119996  249010    86104         0.0
119997  249010   186634         1.0
119998  249010   293818         0.0
119999  249010   262811         1.0

[120000 rows x 3 columns]


,TrackID,Predictor
0,199810_208019,0
1,199810_74139,1
2,199810_9903,0
3,199810_242681,0
4,199810_18515,1
5,199810_105760,1
6,199812_276940,0
7,199812_142408,1
8,199812_130023,1
9,199812_29189,0


In [18]:
df.to_csv('predictions_GBT.csv', index=False)

In [19]:
## Decision Tree
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(train_data)


# Train a DecisionTree model.
dt = DecisionTreeRegressor(featuresCol="indexedFeatures")
# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, dt])

# Train model.  This also runs the indexer.
model = pipeline.fit(train_data)


predictions = model.transform(test_data)
# Select the columns of interest
# Select the columns of interest and concatenate UsrID and TrackID into a new column named USRID_TRACKID
predictions = predictions.withColumn("TrackID_conc", concat(col("UsrID"), lit('_'), col("TrackID")))
predictions1 = predictions.toPandas()

grouped = predictions1.groupby('UsrID')['prediction'].apply(np.array)

# Iterate over each group and select the top 3 indices with the largest values
suggestions = []
for group in grouped:
    indices = np.argsort(group)
    largest_indices = indices[-3:]
    user_suggestions = np.zeros_like(group)
    user_suggestions[largest_indices] = 1
    suggestions.append(user_suggestions)

  # Concatenate the suggestions for all users into a single array
suggestions_array = np.concatenate(suggestions)

# Create a new DataFrame with TrackID, UsrID, and suggestions
suggestions_df = pd.DataFrame({'UsrID': predictions1['UsrID'],'TrackID': predictions1['TrackID'], 'Suggestion': suggestions_array})

# Display the resulting DataFrame
print(suggestions_df)


suggestions_df['UserID_TrackID'] = suggestions_df['UsrID'].astype(str) + '_' + suggestions_df['TrackID'].astype(str)

# Create DataFrame with 'UserID_TrackID' and 'Suggestion' columns
df = pd.DataFrame({'TrackID': suggestions_df['UserID_TrackID'], 'Predictor': suggestions_df['Suggestion']})

# Convert 'Suggestion' column to integer type
df['Predictor'] = df['Predictor'].astype(int)
df.head(12)

         UsrID  TrackID  Suggestion
0       199810   208019         0.0
1       199810    74139         1.0
2       199810     9903         0.0
3       199810   242681         0.0
4       199810    18515         1.0
...        ...      ...         ...
119995  249010    72192         0.0
119996  249010    86104         0.0
119997  249010   186634         1.0
119998  249010   293818         0.0
119999  249010   262811         1.0

[120000 rows x 3 columns]


,TrackID,Predictor
0,199810_208019,0
1,199810_74139,1
2,199810_9903,0
3,199810_242681,0
4,199810_18515,1
5,199810_105760,1
6,199812_276940,0
7,199812_142408,1
8,199812_130023,1
9,199812_29189,0


In [20]:
df.to_csv('predictions_dt.csv', index=False)